<a href="https://colab.research.google.com/github/AkshitAggarwal/TSAI_EVA5B2_Phase1/blob/main/Session_04/S4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports all necessary libraries and methods from Pytorch. 

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# class Net(nn.Module):
#     #Constructor function initializes an object of Net() and defines all the layers. 
#     #Each layer is defined here separately and aren't connected to each other in any way yet. 
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #in_channel: 1,28,28; out_channel: 32,28,28; RF: 3 
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #in_channel: 32,28,28; out_channel: 64,28,28; RF: 5 
#         self.pool1 = nn.MaxPool2d(2, 2) #in_channel: 64,28,28; out_channel: 64,14,14; RF: 10
#         self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #in_channel: 64,14,14; out_channel: 128,14,14; RF: 12
#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #in_channel: 128,14,14; out_channel: 256,14,14; RF: 14
#         self.pool2 = nn.MaxPool2d(2, 2) #in_channel: 256,14,14; out_channel: 256,7,7; RF: 28
#         self.conv5 = nn.Conv2d(256, 512, 3) #in_channel: 256,7,7; out_channel: 512,5,5; RF: 30
#         self.conv6 = nn.Conv2d(512, 1024, 3) #in_channel: 512,5,5; out_channel: 1024,3,3; RF: 32
#         self.conv7 = nn.Conv2d(1024, 10, 3) #in_channel: 1024,3,3; out_channel: 10,1,1; RF: 34

#     #Forward function takes an object and it passes through each layer sequentially. 
#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) #Block 1: conv1 -> conv2 -> maxpool1
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) #Block 2: conv3 -> conv4 -> maxpool2
#         x = F.relu(self.conv6(F.relu(self.conv5(x)))) #Block 3: conv5 -> conv6
#         x = self.conv7(x)
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [3]:
# class Net(nn.Module):
#     #Constructor function initializes an object of Net() and defines all the layers. 
#     #Each layer is defined here separately and aren't connected to each other in any way yet. 
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #in_channel: 1,28,28; out_channel: 32,28,28; RF: 3 
#         self.conv2 = nn.Conv2d(8, 16, 3, padding=1) #in_channel: 32,28,28; out_channel: 64,28,28; RF: 5 
#         self.pool1 = nn.MaxPool2d(2, 2) #in_channel: 64,28,28; out_channel: 64,14,14; RF: 10
#         self.conv3 = nn.Conv2d(16, 32, 3, padding=1) #in_channel: 64,14,14; out_channel: 128,14,14; RF: 12
#         self.conv4 = nn.Conv2d(32, 32, 3, padding=1) #in_channel: 128,14,14; out_channel: 256,14,14; RF: 14
#         self.pool2 = nn.MaxPool2d(2, 2) #in_channel: 256,14,14; out_channel: 256,7,7; RF: 28
#         self.conv5 = nn.Conv2d(32, 10, 3) #in_channel: 256,7,7; out_channel: 512,5,5; RF: 30
#         # self.conv6 = nn.Conv2d(512, 1024, 3) #in_channel: 512,5,5; out_channel: 1024,3,3; RF: 32
#         # self.conv7 = nn.Conv2d(1024, 10, 3) #in_channel: 1024,3,3; out_channel: 10,1,1; RF: 34
#         self.gap = nn.AvgPool2d((5, 5))

#     #Forward function takes an object and it passes through each layer sequentially. 
#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) #Block 1: conv1 -> conv2 -> maxpool1
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) #Block 2: conv3 -> conv4 -> maxpool2
#         # x = F.relu(self.conv6(F.relu(self.conv5(x)))) #Block 3: conv5 -> conv6
#         # x = self.conv7(x)
#         # x = x.view(-1, 10)
#         x = self.gap(self.conv5(x))
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [26]:
class Net(nn.Module):
    #Constructor function initializes an object of Net() and defines all the layers. 
    #Each layer is defined here separately and aren't connected to each other in any way yet. 
    def __init__(self):
        super(Net, self).__init__()
        self.convBlock1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding = 1, bias = False), #in_channel: 1,28,28; out_channel: 16,28,28; RF: 3
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.05)
        )  
        self.convBlock2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding = 1, bias = False), #in_channel: 16,28,28; out_channel: 16,28,28; RF: 5
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.05)
        )
        self.pool1 = nn.MaxPool2d(2, 2) #in_channel: 16,28,28; out_channel: 16,14,14; RF: 10
        self.convBlock3 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding = 1, bias = False), #in_channel: 16,14,14; out_channel: 32,14,14; RF: 12
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.05)
        )
        self.convBlock4 = nn.Sequential(
            nn.Conv2d(32, 32, 3, padding = 1, bias = False), #in_channel: 32,14,14; out_channel: 32,14,14; RF: 14
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.05)
        )
        self.pool2 = nn.MaxPool2d(2, 2) #in_channel: 32,14,14; out_channel: 32,7,7; RF: 28
        self.convBlock5 = nn.Sequential(
            nn.Conv2d(32, 10, 3, bias = False), #in_channel: 32,7,7; out_channel: 10,5,5; RF: 30
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.05)
        )
        self.gap = nn.AvgPool2d((5, 5)) #in_channel: 10,5,5; out_channel: 10,1,1; RF: ?
        self.convBlock6 = nn.Conv2d(10, 10, 1, bias = False) #in_channel: 10,1,1; out_channel: 10,1,1; RF: >32

    #Forward function takes an object and it passes through each layer sequentially. 
    def forward(self, x):
        x = self.convBlock1(x)
        x = self.convBlock2(x)
        x = self.pool1(x)
        x = self.convBlock3(x)
        x = self.convBlock4(x)
        x = self.pool2(x)
        x = self.convBlock5(x)
        x = self.gap(x)
        x = self.convBlock6(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim = -1)

Q1. The global receptive field of our network ends up being more than the size of the image that we input, how will that affect the results?

Q2. What does -1 in Output shape signify?

In [27]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available() #Boolean function checks if cuda(GPU) is available or not.
device = torch.device("cuda" if use_cuda else "cpu") #Assigns cuda or cpu based on availability.
model = Net().to(device) #Converts our model into the respective device.
summary(model, input_size=(1, 28, 28)) #Prints the summary of our model based on an input size. 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
           Dropout-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           2,304
              ReLU-6           [-1, 16, 28, 28]               0
       BatchNorm2d-7           [-1, 16, 28, 28]              32
           Dropout-8           [-1, 16, 28, 28]               0
         MaxPool2d-9           [-1, 16, 14, 14]               0
           Conv2d-10           [-1, 32, 14, 14]           4,608
             ReLU-11           [-1, 32, 14, 14]               0
      BatchNorm2d-12           [-1, 32, 14, 14]              64
          Dropout-13           [-1, 32, 14, 14]               0
           Conv2d-14           [-1, 32,

In [28]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [29]:
from tqdm import tqdm
train_samples = len(train_loader.dataset)
test_samples = len(test_loader.dataset)
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    total_correct = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) #Loading the images and targets on the GPU
        optimizer.zero_grad() 
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss #Calculating the total loss for the epoch. 
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        total_correct += pred.eq(target.view_as(pred)).sum().item()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    print("\nTrain set: Total loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)".format(
              train_loss, total_correct, train_samples, 
             100. * total_correct / train_samples
          ))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [30]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.07472019642591476 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.16it/s]


Train set: Total loss: 238.5517, Accuracy: 52214/60000 (87.0233%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1139, Accuracy: 9716/10000 (97.1600%)


loss=0.06288404017686844 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.75it/s]

...


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0311, Accuracy: 9897/10000 (98.9700%)


loss=0.001021788688376546 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.81it/s]


Train set: Total loss: 5.6713, Accuracy: 59783/60000 (99.6383%)



Test set: Average loss: 0.0250, Accuracy: 9917/10000 (99.1700%)
